### Тонкая настройка и оценка модели бинарной классификации юридических текстов на русском языке с использованием ruBERT 🤖📚

**Цель** 🎯

Целью проекта является разработка модели на основе трансформера BERT, адаптированного для бинарной классификации русскоязычных юридических текстов. Задача модели — определить, содержит ли текст юридические ошибки (класс 1) или же он юридически корректен (класс 0), с целью достижения метрики F1 не менее 0.9.

**Описание** 🧪🔬

Проект охватывает полный цикл машинного обучения, начиная с подготовки и анализа данных, до обучения, валидации и тестирования модели. Основная задача — эффективное использование предобученной модели BERT для русского языка, специально настроенной для задачи классификации текстов. Процесс реализации проекта включает в себя следующие этапы:

**Подготовка данных:**

- Анализ исходных данных, включая загрузку, предварительный анализ содержания и структуры данных.
- Предобработка данных, что включает в себя токенизацию текстов и их преобразование в формат тензоров, подходящих для обработки моделью.
- Разделение данных на обучающую, валидационную и тестовую выборки с учетом баланса классов, что обеспечивает честное и корректное обучение и оценку модели.

**Обучение модели:**

- Настройка архитектуры модели на основе BERT, включая адаптацию модели под конкретную задачу с оптимизацией гиперпараметров.
- Обучение модифицированной модели, используя техники тонкой настройки для максимальной эффективности в задаче классификации.

**Валидация и тестирование модели:**

- Оценка производительности модели на валидационном наборе данных, используя метрики точности, полноты и F1-меры для оценки её способности к обобщению.
- Тестирование модели на отдельном тестовом наборе данных для окончательной верификации её эффективности и надежности предсказаний.

In [1]:
%pip install -q transformers torch

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


import torch
import torch.nn as nn
import transformers
from transformers import AutoModel, BertTokenizer, BertConfig, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

tqdm.pandas()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Загружаем предобученную модель BERT для русского языка.
bert = AutoModel.from_pretrained('.', local_files_only=True)

# Загружаем токенизатор для предобученной модели BERT, предназначенный для русского языка.
tokenizer = BertTokenizer.from_pretrained('.', local_files_only=True)

/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
# Загружаем данные
train_df = pd.read_csv('balance_df.csv')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10000 non-null  object
 1   target  10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


In [6]:
train_df.head()

,text,target
0,РЕШЕНИЕ Именем Российской Федерации 25 июня 2...,0
1,1 РЕШЕНИЕ Именем Российской Федерации 19 окт...,0
2,...,0
3,РЕШЕНИЕ ИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ дата ...,0
4,РЕШЕНИЕ Именем Российской Федерации 21 де...,0


In [4]:
# Разделение train_df на train и temp_df (40%) с учетом стратификации
train_df, temp_df = train_test_split(train_df, test_size=0.4, random_state=42, stratify=train_df['target'])

# Разделение temp_df на val и test (по 50%) с учетом стратификации
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['target'])

# Вывод размеров полученных DataFrame'ов для проверки пропорций
print("Train размер:", train_df.shape)
print("Validation размер:", val_df.shape)
print("Test размер:", test_df.shape)

# Проверка распределения классов после разделения
print("\nРаспределение классов в Train:")
print(train_df['target'].value_counts())
print("\nРаспределение классов в Validation:")
print(val_df['target'].value_counts())
print("\nРаспределение классов в Test:")
print(test_df['target'].value_counts())

Train размер: (6000, 2)
Validation размер: (2000, 2)
Test размер: (2000, 2)

Распределение классов в Train:
1    3000
0    3000
Name: target, dtype: int64

Распределение классов в Validation:
1    1000
0    1000
Name: target, dtype: int64

Распределение классов в Test:
0    1000
1    1000
Name: target, dtype: int64


In [5]:
def split_text_into_segments(text, max_length=510):
    """
    Разбивает текст на сегменты по max_length токенов.
    Обратите внимание, что мы оставляем место для [CLS] и [SEP] токенов.
    
    :param text: Исходный текст.
    :param max_length: Максимальная длина сегмента без учета [CLS] и [SEP].
    :return: Список сегментов текста.
    """
    # Токенизируем текст с учетом возможности разбиения на слова, а не на subwords
    tokens = tokenizer.tokenize(text)
    segments = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return [" ".join(segment) for segment in segments]

In [6]:
def prepare_dataset(df):
    """
    Преобразует датафрейм, разбивая тексты на фрагменты, дублируя соответствующие метки,
    и добавляя позиционные идентификаторы для каждого фрагмента.
    
    :param df: Исходный DataFrame с текстами и метками.
    :return: Новый DataFrame, где каждая строка соответствует фрагменту исходного текста,
             с добавлением позиционных идентификаторов.
    """
    new_records = []
    for _, row in df.iterrows():
        segments = split_text_into_segments(row['text'])
        for i, segment in enumerate(segments):  # Использование enumerate для получения индекса фрагмента
            new_records.append({
                'text': segment,
                'target': row['target'],
                'position_id': i  # Добавление позиционного идентификатора
            })
    return pd.DataFrame(new_records)

# Применяем преобразование к тренировочному, валидационному и тестовому наборам
train_df = prepare_dataset(train_df)
val_df   = prepare_dataset(val_df)
test_df  = prepare_dataset(test_df)

In [7]:
# Конвертация текста и меток из датафреймов в строки и целевые переменные для обучения, валидации и тестирования.
train_text, train_labels = train_df[['text', 'position_id']].astype('str'), train_df['target']
val_text, val_labels = val_df[['text', 'position_id']].astype('str'), val_df['target']
test_text, test_labels = test_df[['text', 'position_id']].astype('str'), test_df['target']

Токенизируем текста, передадим в тензоры и загрузим в функцию DataLoader, которая будет по частям подавать наши данные для обучения и валидации в модель:

In [17]:
# Модифицируем функцию токенизации, чтобы включить логику обнуления position_id для новых текстов
def tokenize_and_preserve_labels(df, tokenizer, max_length=512):
    """
    Токенизирует тексты из датафрейма и сохраняет соответствующие метки классов,
    учитывая при этом позиционные идентификаторы для каждого фрагмента текста.
    
    Данная функция предназначена для подготовки данных к обучению или тестированию модели,
    которая требует не только токенов и масок внимания, но и позиционных идентификаторов
    для каждого токена, чтобы учесть порядок фрагментов в исходном тексте.
    
    Параметры:
        df (pandas.DataFrame): Датафрейм, содержащий тексты и метки классов, а также
                               позиционные идентификаторы для каждого фрагмента текста.
        tokenizer (transformers.PreTrainedTokenizer): Токенизатор, предоставляемый библиотекой
                                                       transformers, для преобразования текста в токены.
        max_length (int, optional): Максимальная длина токенизированной последовательности.
                                    По умолчанию равна 512.
    
    Возвращает:
        tuple: Кортеж, содержащий следующие элементы:
            - input_ids (torch.Tensor): Тензор с идентификаторами токенов для каждого текста.
            - attention_masks (torch.Tensor): Тензор с масками внимания для каждого текста.
            - position_ids (torch.Tensor): Тензор с позиционными идентификаторами для каждого фрагмента текста.
            - labels (torch.Tensor): Тензор с метками классов для каждого текста.
    """
    input_ids = []
    attention_masks = []
    position_ids = []
    labels = []

    # Обнуляем position_id для каждого нового текста
    current_position_id = 0
    previous_text = ""
    
    for index, row in df.iterrows():
        text = row['text']
        label = row['target']
        position_id = row['position_id']

        # Проверяем, начался ли новый текст
        if position_id == 0:
            current_position_id = 0  # Обнуляем position_id для нового текста

        # Токенизация текста
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # Добавляем [CLS] и [SEP]
            max_length=max_length,  # Ограничиваем длину входной последовательности
            pad_to_max_length=True,  # Добавляем паддинг до максимальной длины
            return_attention_mask=True,  # Возвращаем маску внимания
            truncation=True  # Обрезаем до максимальной длины
        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        position_ids.append([current_position_id] * max_length)  # Генерируем position_ids
        labels.append(label)
        
        current_position_id += 1  # Увеличиваем position_id для следующего фрагмента

    # Преобразуем списки в тензоры PyTorch
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    position_ids = torch.tensor(position_ids)
    labels = torch.tensor(labels, dtype=torch.long)

    return input_ids, attention_masks, position_ids, labels

# Размер пакета для обучения.
batch_size = 8

# Применяем функцию к датасетам
input_ids_train, attention_masks_train, position_ids_train, labels_train = tokenize_and_preserve_labels(train_df, tokenizer)
input_ids_val, attention_masks_val, position_ids_val, labels_val = tokenize_and_preserve_labels(val_df, tokenizer)
input_ids_test, attention_masks_test, position_ids_test, labels_test = tokenize_and_preserve_labels(test_df, tokenizer)

# Создаем DataLoader'ы
train_data = TensorDataset(input_ids_train, attention_masks_train, position_ids_train, labels_train)
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=batch_size)

val_data = TensorDataset(input_ids_val, attention_masks_val, position_ids_val, labels_val)
val_dataloader = DataLoader(val_data, sampler=SequentialSampler(val_data), batch_size=batch_size)

test_data = TensorDataset(input_ids_test, attention_masks_test, position_ids_test, labels_test)
test_dataloader = DataLoader(test_data, sampler=SequentialSampler(test_data), batch_size=batch_size)

In [ ]:
# Сохраняем DataLoader'ы
with open('train_dataloader.pkl', 'wb') as f:
    pickle.dump(train_dataloader, f)

with open('val_dataloader.pkl', 'wb') as f:
    pickle.dump(val_dataloader, f)

with open('test_dataloader.pkl', 'wb') as f:
    pickle.dump(test_dataloader, f)

In [5]:
# Загружаем DataLoader'ы
with open('train_dataloader.pkl', 'rb') as f:
    train_dataloader = pickle.load(f)

with open('val_dataloader.pkl', 'rb') as f:
    val_dataloader = pickle.load(f)

with open('test_dataloader.pkl', 'rb') as f:
    test_dataloader = pickle.load(f)

In [6]:
class BERT_Arch(nn.Module):
    """
    Модифицированная архитектура модели на основе BERT, включающая слой внимания для агрегации фрагментов текста
    и позиционные эмбеддинги для учета порядка фрагментов.
    
    Атрибуты:
        bert (transformers.BertModel): Предобученная модель BERT.
        attention (torch.nn.Sequential): Слой внимания, который вычисляет веса для фрагментов текста.
        position_embeddings (torch.nn.Embedding): Позиционные эмбеддинги для учета порядка фрагментов.
        classifier (torch.nn.Linear): Классификатор для бинарной классификации.
        dropout (torch.nn.Dropout): Слой дропаута.
        relu (torch.nn.ReLU): Функция активации ReLU.
        
    Методы:
        forward(input_ids, attention_mask, position_ids): Выполняет прямой проход модели.
    """
    
    def __init__(self, bert, config):
        super(BERT_Arch, self).__init__()
        self.bert = bert      # Инициализация модели BERT
        self.config = config  # Конфигурация модели BERT

        # Слой внимания для агрегации информации из различных фрагментов текста
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),  # Преобразование размерности скрытого слоя
            nn.Tanh(),                                # Применение функции активации Tanh
            nn.Linear(512, 1),                        # Сужение до одного выхода для весов внимания
            nn.Softmax(dim=1)                         # Softmax для получения весов внимания
        )
        
        # Позиционные эмбеддинги для учета порядка фрагментов
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, self.config.hidden_size)
        
        self.classifier = nn.Linear(self.config.hidden_size, 1)  # Классификатор для бинарной классификации
        self.dropout = nn.Dropout(0.1)                           # Слой дропаута
        self.relu = nn.ReLU()                                    # Функция активации ReLU

    def forward(self, input_ids, attention_mask, position_ids):
        """
        Выполняет прямой проход модели.

        Параметры:
            input_ids (torch.Tensor): Тензор с идентификаторами токенов.
            attention_mask (torch.Tensor): Тензор с маской внимания.
            position_ids (torch.Tensor): Тензор с позиционными идентификаторами для каждого фрагмента.

        Возвращает:
            torch.Tensor: Логиты бинарной классификации для каждого примера.
        """
        outputs = self.bert(input_ids, attention_mask=attention_mask)  # Получение выхода из BERT
        sequence_output = outputs[0]                                   # Последний скрытый слой BERT

        # Добавление позиционных эмбеддингов
        position_embeddings = self.position_embeddings(position_ids)  # Получение позиционных эмбеддингов
        sequence_output += position_embeddings                        # Сложение позиционных эмбеддингов с выходом BERT

        # Применение слоя внимания
        attention_weights = self.attention(sequence_output[:, 0, :])  # Вычисление весов внимания
        # Расширяем размерность весов внимания и применяем их к каждому вектору в sequence_output
        attention_weights = attention_weights.unsqueeze(-1).expand_as(sequence_output)
        weighted_sum = torch.sum(attention_weights * sequence_output, dim=1)  # Агрегация с весами внимания

        logits = self.classifier(self.dropout(self.relu(weighted_sum)))  # Применение классификатора
        return logits

In [7]:
# Получаем конфигурацию модели из загруженной модели BERT
config = bert.config

# Инициализация модифицированной архитектуры с загруженной моделью BERT и ее конфигурацией
model = BERT_Arch(bert, config)

# Перемещение модели на устройство (например, на GPU, если доступно)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Модель успешно инициализирована и перемещена на устройство:", device)

Модель успешно инициализирована и перемещена на устройство: cuda


In [ ]:
def train(model, dataloader, optimizer, criterion, device):
    """
    Функция обучения модели.

    Аргументы:
        model (torch.nn.Module): Модель для обучения.
        dataloader (DataLoader): DataLoader для обучающего набора данных.
        optimizer (torch.optim.Optimizer): Оптимизатор для обновления весов модели.
        criterion (torch.nn.Module): Функция потерь.
        device (torch.device): Устройство, на котором производится вычисление (CPU или GPU).

    Возвращает:
        float: Среднее значение потерь на обучающем наборе данных.
    """
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, position_ids, labels = batch
        labels = labels.float().unsqueeze(1)  # Преобразование меток и добавление измерения

        optimizer.zero_grad()
        
        # Важно: обновите вызов модели, чтобы он включал position_ids
        outputs = model(input_ids, attention_mask, position_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    """
    Функция оценки модели.

    Аргументы:
        model (torch.nn.Module): Модель для оценки.
        dataloader (DataLoader): DataLoader для валидационного набора данных.
        criterion (torch.nn.Module): Функция потерь.
        device (torch.device): Устройство для вычислений.

    Возвращает:
        float: Среднее значение потерь на валидационном наборе данных.
    """
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, position_ids, labels = batch
            labels = labels.float().unsqueeze(1)  # Преобразование меток и добавление измерения

            # Важно: обновите вызов модели, чтобы он включал position_ids
            outputs = model(input_ids, attention_mask, position_ids)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
    return total_loss / len(dataloader)

# Создание оптимизатора и функции потерь
optimizer = AdamW(model.parameters(), lr=1e-3)
# criterion = nn.CrossEntropyLoss().to(device)
criterion = nn.BCEWithLogitsLoss().to(device)


# Основной цикл обучения
epochs = 4

for epoch in range(epochs):
    print(f'\nEpoch {epoch + 1}/{epochs}')
    train_loss = train(model, train_dataloader, optimizer, criterion, device)
    val_loss = evaluate(model, val_dataloader, criterion, device)
    print(f'Training loss: {train_loss}')
    print(f'Validation loss: {val_loss}')

In [9]:
model.eval()  # Перевод модели в режим оценки
total_test_loss = 0
predictions, true_labels = [], []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, position_ids, labels = batch

        outputs = model(input_ids, attention_mask, position_ids)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        
        total_test_loss += loss.item()
        
        preds = torch.sigmoid(outputs).cpu().detach().numpy()  # Применяем сигмоиду для получения вероятностей
        batch_labels = labels.cpu().detach().numpy()
        
        predictions.extend(preds)
        true_labels.extend(batch_labels)

Testing: 100%|██████████| 1425/1425 [01:33<00:00, 15.31it/s]


In [1]:
# Преобразование вероятностей в бинарные предсказания
threshold = 0.5
binary_preds = [1 if x > threshold else 0 for x in predictions]

# Рассчитываем метрики
accuracy = accuracy_score(true_labels, binary_preds)
precision = precision_score(true_labels, binary_preds)
recall = recall_score(true_labels, binary_preds)
f1 = f1_score(true_labels, binary_preds)

print(f"Accuracy: {accuracy}") 
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.8996052285288183
Precision: 0.8996052285288183
Recall: 0.9112874779541447
F1 Score: 0.9053497942386817
